In [1]:
from znh5md.templates import LammpsH5MD
from zinchub import DataHub

In [2]:
NaCl = DataHub(url="https://github.com/zincware/DataHub/tree/main/NaClH5MD")

'lammps_npt.h5'

In [3]:
traj = LammpsH5MD(NaCl.get_file())

In [7]:
print(traj.position[:5, :10].shape)

(5, 10, 3)


In [5]:
for position in traj.position.get_dataset().batch(16):
    print(position)
    break

{'step': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([    0,  1000,  2000,  3000,  4000,  5000,  6000,  7000,  8000,
        9000, 10000, 11000, 12000, 13000, 14000, 15000], dtype=int32)>, 'time': <tf.Tensor: shape=(16,), dtype=float64, numpy=
array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15.])>, 'value': <tf.Tensor: shape=(16, 1000, 3), dtype=float64, numpy=
array([[[12.68      , 12.76      , 22.45      ],
        [14.13      , 18.42      , 15.32      ],
        [29.63      ,  9.73      ,  6.4       ],
        ...,
        [25.7       , 31.43      , 10.21      ],
        [12.87      ,  6.86      , 31.61      ],
        [14.98      , 25.42      , 26.76      ]],

       [[14.32642164, 14.38138961, 21.49369015],
        [13.0392567 , 17.96638991, 17.42442922],
        [30.08765406, 10.68043946,  5.9443841 ],
        ...,
        [24.16762117, 30.10362162, 10.98561193],
        [11.18806231,  7.56374404, 29.65563923],
        [15.52416291,

In [10]:
%%time
for position in traj.position.value.get_dataset(selection=slice(500)).batch(32):
    print(position.shape)

(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(9, 500, 3)
CPU times: user 232 ms, sys: 32.5 ms, total: 265 ms
Wall time: 425 ms


In [15]:
%%time
for position in traj.position.value.get_dataset(selection=slice(500), prefetch=32).batch(32):
    print(position.shape)

(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(32, 500, 3)
(9, 500, 3)
CPU times: user 57.1 ms, sys: 8.46 ms, total: 65.5 ms
Wall time: 66.1 ms


In [17]:
traj.velocity[:5, :3].shape

(5, 3, 3)